In [1]:
from PIL import Image
import numpy as np
import tensorflow as tf
import os
import cv2
import random

from tensorflow import keras
from tensorflow.keras import layers, regularizers, optimizers
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import Callback
from tensorflow import Tensor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


import matplotlib.pyplot as plt


In [2]:
%%time
AIimages = []
new_size = (100, 100)
AI_path = "D:\AI vs Real Classification\AiArtData\AiArtData"

for file in os.listdir(AI_path):
    if file.endswith(".png") or file.endswith(".jpg"):
        img = Image.open(os.path.join(AI_path, file)).resize(new_size)
        npArray = np.array(img)
        if npArray.shape == (100, 100, 3): AIimages.append(npArray)
        
print(f'Number of AI images after data cleaning (removing black & white images): {len(AIimages)}')
        
RealImages = []
Real_path = "D:\AI vs Real Classification\RealArt\RealArt"

for file in os.listdir(Real_path):
    if file.endswith(".png") or file.endswith(".jpg"):
        img = Image.open(os.path.join(Real_path, file)).resize(new_size)
        npArray = np.array(img)
        if npArray.shape == (100, 100, 3): RealImages.append(npArray)
        
print(f'Number of Real images after data cleaning (removing black & white images): {len(RealImages)}')
        
AILabel = []
RealLabel = []
for _ in range(len(AIimages)): AILabel.append(0)
for _ in range(len(RealImages)): RealLabel.append(1)

Number of AI images after data cleaning (removing black & white images): 450
Number of Real images after data cleaning (removing black & white images): 385
CPU times: total: 7.69 s
Wall time: 15.4 s


In [3]:
%%time
datagen = ImageDataGenerator(
    rotation_range=90,      # Rotate images randomly by up to 20 degrees
    width_shift_range=0.2,  # Shift images horizontally by up to 20% of the width
    height_shift_range=0.2, # Shift images vertically by up to 20% of the height
    shear_range=0.2,        # Shear transformations
    zoom_range=0.2,         # Zoom in/out randomly by up to 20%
    horizontal_flip=True,  # Flip images horizontally
    fill_mode='nearest'     # Fill in missing pixels with the nearest value
)
# Generate augmented images
augmented_AIimages = []
augmented_RealImages = []

for img in AIimages:
    img = np.expand_dims(img, axis=0)
    for _ in range(5):
        augmented_img = datagen.flow(img, batch_size=1).next()[0]
        augmented_AIimages.append(augmented_img)

for img in RealImages:
    img = np.expand_dims(img, axis=0)
    for _ in range(5):
        augmented_img = datagen.flow(img, batch_size=1).next()[0]
        augmented_RealImages.append(augmented_img)
        
print(len(augmented_AIimages))
print(len(augmented_RealImages))
        
augmented_AILabel = []
augmented_RealLabel = []
    
for _ in range(len(augmented_AIimages)):
    augmented_AILabel.append(0)
for _ in range(len(augmented_RealImages)):
    augmented_RealLabel.append(1)

2250
1925
CPU times: total: 4.78 s
Wall time: 5.13 s


In [4]:
x_train = np.array(AIimages + RealImages + augmented_AIimages + augmented_RealImages)
y_train = np.array(AILabel + RealLabel + augmented_AILabel + augmented_RealLabel)

y_train = keras.utils.to_categorical(y_train, 2)

print(x_train.shape)
print(y_train.shape)

(5010, 100, 100, 3)
(5010, 2)


In [5]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]
x_train, y_train = unison_shuffled_copies(x_train, y_train)

In [22]:
def build_model():
    print('now training')
    model = keras.Sequential([
        keras.Input(shape=(100, 100, 3)),
        layers.Conv2D(200, (7, 7)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Dropout(0.3),
        layers.Conv2D(150, (5, 5)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Dropout(0.3),
        layers.Conv2D(100, (3, 3)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Dropout(0.3),
        layers.Conv2D(50, (3, 3)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Dropout(0.3),
        layers.Flatten(),
        layers.Dense(50, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(20, activation='relu'),
        layers.Dropout(0.1),
        layers.Dense(2, activation='softmax'),
    ])
    
    return model

def train(sp = 'model.h5', epochs=60):
    batch_size = 10
    model = build_model()
    model.compile(loss="binary_crossentropy", optimizer = 'adam', metrics=["accuracy"])
    model.fit(
        x_train, 
        y_train,
        batch_size = batch_size,
        epochs = epochs,
        validation_split = 0.2)
    #model.evaluate(x_test, y2_test, verbose=1)
    model.save(sp)
    
train()

now training
Epoch 1/60
401/401 [==============================] - 7s 15ms/step - loss: 0.6894 - accuracy: 0.5833 - val_loss: 0.6658 - val_accuracy: 0.6407
Epoch 2/60
401/401 [==============================] - 6s 15ms/step - loss: 0.6614 - accuracy: 0.6175 - val_loss: 0.6790 - val_accuracy: 0.5679
Epoch 3/60
401/401 [==============================] - 6s 15ms/step - loss: 0.6462 - accuracy: 0.6390 - val_loss: 0.6316 - val_accuracy: 0.6477
Epoch 4/60
401/401 [==============================] - 6s 15ms/step - loss: 0.6465 - accuracy: 0.6400 - val_loss: 0.6290 - val_accuracy: 0.6437
Epoch 5/60
401/401 [==============================] - 6s 15ms/step - loss: 0.6359 - accuracy: 0.6455 - val_loss: 0.6286 - val_accuracy: 0.6427
Epoch 6/60
401/401 [==============================] - 6s 15ms/step - loss: 0.6308 - accuracy: 0.6462 - val_loss: 0.6371 - val_accuracy: 0.6248
Epoch 7/60
401/401 [==============================] - 6s 15ms/step - loss: 0.6237 - accuracy: 0.6529 - val_loss: 0.6473 - val_acc